# GEOCODING
### Developed by Bernardo Puente 


Last update: 10-Dec-2020

## Context

Geocoding is the process of converting addresses (like "1600 Amphitheatre Parkway, Mountain View, CA") into geographic coordinates (like latitude 37.423021 and longitude -122.083739), which you can use to place markers on a map, or position the map.

<img src="https://www.ptvgroup.com/fileadmin/_processed_/a/9/csm_01_PTV_xServer_Infograph_Geocoding_274fe6bdc6.jpg" alt="drawing" width="500">

For more info on the GOOGLE MAPS GEOCODING API visit: https://developers.google.com/maps/documentation/geocoding/intro

**Important:** Recall you must have a Google Cloud Platform account to run this code with your given API Key. 

**Important:** Google Maps GEOCODING API has a cost of 1000 Directions = 5.00 USD. Google Cloud Platform offers a monthly free ammount of 200 USD, so you have free 40 000 directions to run. Further than thay you will be charged.

## Input
This code takes as an input an excel file containing addressess which will be the input for the GEOCODING API to convert into coordinates.

**Important:** The preferred address format to obtain the best results is the standard address format of the postal system of each country. Modifications to this will reduce the accuracy of the results or increase the ammount of errors. For Ecuador the preferred address format is: "Main Street **&** Secondary Street, City Country". An example of a valid input address is "DE GOYA & FRA ANGELICO, QUITO ECUADOR". Using upper or lower case is not relevant. **DO NOT INCLUDE NUMERATION SUCH AS "E23-2" OR "S/N"** since this will reduce the accuracy of the results for the particular case of Ecuador. Consult your local postal system for your best address format.

## Output
This code will output a similar file as the input file, with the original addressess, plus additional columns containing the geographic resulting coordinates **and** status of the geocoding query (e.g. succesfull queries, errors, etc).

# Code Start:

Import all libraries needed:

* pandas for input and output dataframe handling
* numpy to handle google maps API output in list and dict forms
* googlemaps to activate Geocoding API requests
* logging to access logger messages
* time to measure code execution times


In [1]:
import pandas as pd
import numpy as np
import googlemaps
import logging
import time

Set up logger to show desired messages in console as warnings.

In [2]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

Specify api key name and input/output files.

In [4]:
api_key='YourAPI_Key_here' #Your valid API key here
file_input='INPUT.xlsx' 
file_output='OUTPUT.xlsx'

Read input file and identify data structure.

In [5]:
direcciones=pd.read_excel(file_input)
#direcciones = pd.read_csv(file_input, sep=';', encoding='latin-1') #You can import 
logger.warning('CHECK INSERTED ADDRESSESS. MAKE SURE ADDRESSES READ SPECIAL CHARACTERS CORRECTLY SUCH AS Á,É,Ñ,ä, IF THESE CHARACTERS ARE NOT WELL INSERTED TRY TO INCLUDE AN ENCODING SUCH AS "latin-1" IN THE PREVIOUS FILE READING PROCEDURE. WRONG ADDRESSESS ARE INCLUDED TO SHOW ERROR HANDLING IN THE NEXT STEPS.')
direcciones.head()

CHECK INSERTED ADDRESSESS. MAKE SURE ADDRESSES READ SPECIAL CHARACTERS CORRECTLY SUCH AS Á,É,Ñ,ä, IF THESE CHARACTERS ARE NOT WELL INSERTED TRY TO INCLUDE AN ENCODING SUCH AS "latin-1" IN THE PREVIOUS FILE READING PROCEDURE. WRONG ADDRESSESS ARE INCLUDED TO SHOW ERROR HANDLING IN THE NEXT STEPS.


,DIRECCION
0,"AV. BOYACÁ & LATACUNGA, QUITO"
1,"10 DE DICIEMBRE & VIA ALBORNOZ, RUMINAHUI"
2,"DE GOYA & FRA ANGELICO, QUITO"
3,"PEDRO DE ALVARADO & JOSE FIGUEROA, QUITO"
4,"GONZALO PIZARRO & GONZALO MENESES, QUITO"


Convert all addresses to be geocoded to list for better accessing to that specific data. First 5 addresses are shown. Also, recall addresses do not include the country, so we include the country to each address to mantain the suggested address format.

In [6]:
addresses=(direcciones['DIRECCION']+' ECUADOR').tolist()
addresses[:5]

['AV. BOYACÁ & LATACUNGA, QUITO ECUADOR',
 '10 DE DICIEMBRE & VIA ALBORNOZ, RUMINAHUI ECUADOR',
 'DE GOYA & FRA ANGELICO, QUITO ECUADOR',
 'PEDRO DE ALVARADO & JOSE FIGUEROA, QUITO ECUADOR',
 'GONZALO PIZARRO & GONZALO MENESES, QUITO ECUADOR']

Test the API functionality. Check test address result and errors.

In [7]:
gmaps = googlemaps.Client(key=api_key)
start = time.time()

try:
    gmaps = googlemaps.Client(key=api_key)
    #Change the following address to a known one, or leave if test with first address and validate the output result.
    sample_address=addresses[0]
    test= gmaps.geocode(sample_address) 
    end=(time.time() - start)
    logger.warning('Geocoded results for: '+sample_address+' are: '+str(test[0]['geometry']['location']))
    logger.warning('Verify the obtained result before proceding.')
    logger.warning('Time elapsed to complete execution: '+str(end)+' seconds.')
    logger.warning('GOOD TO GO. Beware of ammount of geocoding points. 1000 Directions = $5.00 USD.')
except:
    logger.warning('AN ERROR OCCURRED, REVISE CODE, API KEY VALIDITY, OR INSERTED TEST ADDRESS.')

Geocoded results for: AV. BOYACÁ & LATACUNGA, QUITO ECUADOR are: {'lat': -0.209241, 'lng': -78.395484}
Verify the obtained result before proceding.
Time elapsed to complete execution: 0.7124838829040527 seconds.
GOOD TO GO. Beware of ammount of geocoding points. 1000 Directions = $5.00 USD.


Run the Geocoding procedure for each address in the list. Outputs are exported to a .csv file containing (latitude, longitude) results, Person ID, Person Name, Input Address, Output Address.

In [8]:
start_time = time.time() #we start recording the clock to then obtain the duration of the procedure. 10K Addressess will take 5 hours approx.
results=[] #Create an ampty list where the results will be saved
i=0 #This counter will work as a control for the ammount of addressess completed.
errores=0 #We create this variable to keep track of the ammount of errors in our desired list.
for value in addresses: #we use each address value as the iterable. The iterable "value" is the address itself
    try:
        # Geocoding the address corresponding to the value in the loop
        geocode_result = gmaps.geocode(value)
        
        # Extract only lat and long WHEN THERE IS A VALID RESULT FOUND from the geocode procedure.
        if geocode_result!=[]: #If geocode_result=[] (e.g. is empty), there is no result, then skip to next segment. If there is a result, the result is recorded in the results list.
            results.append(geocode_result[0]['geometry']['location'])
            logger.warning('Geocoded: '+value+'. Added address and details to list dict record.')
            results[i].update({'Input_Address': value})
            results[i].update({'Formated_Address': geocode_result[0]['formatted_address']})
            results[i].update({'Status': 'OK'})
        
        # If there is no result found (e.g. empty result []) append 0.0,0.0 as coordinates and NO RESULTS message
        elif geocode_result==[]:
            results.append({'lat': 0.0,
                            'lng': 0.0,
                            'Input_Address': value,
                            'Formated_Address': 'NO RESULTS',
                            'Status': 'NO RESULT'})
            logger.warning('NO RESULTS ')
            
    # If there is a error of another nature (e.g. strange characters as input, error connecting to API, etc), skip the address and record an error message
    except: 
        results.append({'lat': 0.0,
                        'lng': 0.0,
                        'Input_Address': value,
                        'Formated_Address': 'ERROR IN THE ADDRESS FORMAT, SKIPPED TO NEXT POINT',
                        'Status': 'ERROR'})
        errores+=1
        logger.warning('ERROR IN THE ADDRESS FORMAT, SKIPPED TO NEXT POINT')
        
    # Tell the user the number of completed geocoded addresses every 100 addressess
    i=i+1
    if i%100==0:
        logger.warning(str(i)+' ADDRESSES COMPLETED! ')

#Show the final details of the procedure such as completion of the process, the ammount of errors found, the ammount and time of geocoded results.
logger.warning('ALL DIRECTIONS GEOCODED SUCCESSFULLY')
logger.warning('NUMBER OF ERRORS FOUND: '+str(errores)+'. BEWARE TO SOLVE THEM IN THE OUTPUT FILE!')
pd.DataFrame(results).to_excel(file_output, encoding='utf-8')
logger.warning('OUTPUT FILE GENERATED')
end_time=(time.time() - start_time)
logger.warning('TIME TO GEOCODE '+str(i)+' ADDRESSES: '+str(end_time/60)+' MINUTES, OR '+str(end_time)+' SECONDS')
logger.warning('PROCESS FINISHED.')

Geocoded: AV. BOYACÁ & LATACUNGA, QUITO ECUADOR. Added address and details to list dict record.
Geocoded: 10 DE DICIEMBRE & VIA ALBORNOZ, RUMINAHUI ECUADOR. Added address and details to list dict record.
Geocoded: DE GOYA & FRA ANGELICO, QUITO ECUADOR. Added address and details to list dict record.
Geocoded: PEDRO DE ALVARADO & JOSE FIGUEROA, QUITO ECUADOR. Added address and details to list dict record.
Geocoded: GONZALO PIZARRO & GONZALO MENESES, QUITO ECUADOR. Added address and details to list dict record.
NO RESULTS 
Geocoded: QUEVEDO & AMBATO, RUMINAHUI ECUADOR. Added address and details to list dict record.
NO RESULTS 
ALL DIRECTIONS GEOCODED SUCCESSFULLY
NUMBER OF ERRORS FOUND: 0. BEWARE TO SOLVE THEM IN THE OUTPUT FILE!
OUTPUT FILE GENERATED
TIME TO GEOCODE 8 ADDRESSES: 0.07352201541264852 MINUTES, OR 4.411320924758911 SECONDS
PROCESS FINISHED.


## End of code.